<h2>About this Exercise</h2>
<p>In the preceding activity, you derived a Euclidean distance matrix. Now that you have calculated the distance between points in terms of matrix operations, you are ready to write an efficient program that leverages NumPy's optimized functions. In this code exercise, rather than using loops, you will write a function to compute Euclidean distances between sets of vectors using NumPy functions.</p>

<h3>Evaluation</h3>

<p><strong>You must complete this exercise in order to unlock the final project in this module. Your score on this assignment will not be included in the final grade calculation.</strong><p>

<p>You are expected to write code where you see <em># YOUR CODE HERE</em> within the cells of this notebook. Not all cells will be graded; code input cells followed by cells marked with <em>#Autograder test cell</em> will be graded. Upon submitting your work, the code you write at these designated positions will be assessed using an "autograder" that will run all test cells to assess your code. You will receive feedback from the autograder that will identify any errors in your code. Use this feedback to improve your code if you need to resubmit. Be sure not to change the names of any provided functions, classes, or variables within the existing code cells, as this will interfere with the autograder. Also, remember to execute all code cells sequentially, not just those you’ve edited, to ensure your code runs properly.</p>
    
<p>You can resubmit your work as many times as necessary before the submission deadline. If you experience difficulty or have questions about this exercise, use the Q&A discussion board to engage with your peers or seek assistance from the instructor.<p>

<p>Before starting your work, please review <a href="https://s3.amazonaws.com/ecornell/global/eCornellPlagiarismPolicy.pdf">eCornell's policy regarding plagiarism</a> (the presentation of someone else's work as your own without source credit).</p>

<h3>Submit Code for Autograder Feedback</h3>

<p>Once you have completed your work on this notebook, you will submit your code for autograder review. Follow these steps:</p>

<ol>
  <li><strong>Save your notebook.</strong></li>
  <li><strong>Mark as Completed —</strong> In the blue menu bar along the top of this code exercise window, you’ll see a menu item called <strong>Education</strong>. In the <strong>Education</strong> menu, click <strong>Mark as Completed</strong> to submit your code for autograder/instructor review. This process will take a moment and a progress bar will show you the status of your submission.</li>
	<li><strong>Review your results —</strong> Once your work is marked as complete, the results of the autograder will automatically be presented in a new tab within the code exercise window. You can click on the assessment name in this feedback window to see more details regarding specific feedback/errors in your code submission.</li>
  <li><strong>Repeat, if necessary —</strong> The Jupyter notebook will always remain accessible in the first tabbed window of the exercise. To reattempt the work, you will first need to click <strong>Mark as Uncompleted</strong> in the <strong>Education</strong> menu and then proceed to make edits to the notebook. Once you are ready to resubmit, follow steps one through three. You can repeat this procedure as many times as necessary.</li>
</ol>

<h2>Import NumPy and Check Python Version</h2>

First, you must import NumPy. Let's also check our version of Python. We've added the code for you for this first step.

In [25]:
import sys
import numpy as np # Numpy is Python's built in library for matrix operations.
from pylab import * 
sys.path.append('/home/codio/workspace/.guides/hf')
from helper import *
print('You\'re running python %s' % sys.version.split(' ')[0])

You're running python 3.6.8



<h2> Euclidean Distances in Python </h2>

<p>Many machine learning algorithms access their input data primarily through pairwise (Euclidean) distances, therefore it is important that we have a fast function that computes pairwise distances of input vectors.</p>
<p>Assume we have $n$ row data vectors $\mathbf{x_1}, \dots, \mathbf{x_n} \in \mathcal{R}^d$ and $m$ row vectors $\mathbf{z_1}, \dots, \mathbf{z_m} \in \mathcal{R}^d$. With these data vectors, let us define two matrices $X = \left[ \mathbf{x_1}^\top, \dots, \mathbf{x_n}^\top \right]^\top \in \mathcal{R}^{n \times d}$, where the $i^{th}$ row is vector $\mathbf{x_i}$, and $Z = \left[ \mathbf{z_1}^\top, \dots, \mathbf{z_m}^\top \right]^\top \in \mathcal{R}^{m \times d}$.We want a distance function that takes as input these two matrices $X$ and $Z$ and outputs a matrix $D\in{\cal R}^{n\times m}$, where 
	$$D_{ij}=\sqrt{(\mathbf{x_i}-\mathbf{z_j})(\mathbf{x_i}-\mathbf{z_j})^\top}.$$
</p>

A naïve implementation to compute pairwise distances may look like the code below:

In [16]:
def l2distanceSlow(X,Z=None):
    if Z is None:
        Z = X
    
    n, d = X.shape     # dimension of X
    m= Z.shape[0]   # dimension of Z
    D=np.zeros((n,m)) # allocate memory for the output matrix
    for i in range(n):     # loop over vectors in X
        for j in range(m): # loop over vectors in Z
            D[i,j]=0.0; 
            for k in range(d): # loop over dimensions
                D[i,j]=D[i,j]+(X[i,k]-Z[j,k])**2; # compute l2-distance between the ith and jth vector
            D[i,j]=np.sqrt(D[i,j]); # take square root
    return D

Please read through the code above carefully and make sure you understand it. It is perfectly correct and will produce the correct result... eventually. To see what is wrong, try running the <code>l2distanceSlow</code> code below on an extremely small matrix <code>X</code>.


In [17]:
X=np.random.rand(700,100)
print("Running the naive version, please wait...")
%time Dslow=l2distanceSlow(X)

Running the naive version, please wait...
CPU times: user 34.8 s, sys: 0 ns, total: 34.8 s
Wall time: 34.8 s


This code defines some random data in $X$ and computes the corresponding distance matrix $D$. The <em>%time</em> statement determines how long this code takes to run. This implementation is much too slow for such a simple operation on a small amount of data, and writing code like this to deal with matrices in this course will result in code that takes <strong>days</strong> to run.

<strong>As a general rule, you should avoid tight loops at all cost.</strong> As you will see in the remainder of this exercise, you can do much better by performing bulk matrix operations using the NumPy package, which calls highly optimized compiled code behind the scenes.





<h2> Efficient Programming with NumPy </h2>

<p>Although there is an execution overhead per line in Python, matrix operations are optimized and fast. In order to successfully program in this course, you need to free yourself from "for-loop" thinking and start thinking in terms of matrix operations. Python for scientific computing can be very fast if almost all the time is spent on a few heavy duty matrix operations. In this exercise, you will transform the function above into a few matrix operations <em>without any loops at all.</em> </p> 

<p>The key to efficient programming in Python for machine learning in general is to think about it in terms of mathematics and not in terms of loops. </p>

<h2>Exercises</h2>

<p>In the following three exercises, you'll take the steps necessary to implement the euclidean distance function without loops.</p>

<h3>Exercise 1: Inner-Product Matrix</h3>

<p>Show that the Inner-Product Matrix (Gram matrix) can be expressed in terms of pure matrix multiplication.

$$	G_{ij}=\mathbf{x}_i\mathbf{z}_j^\top $$

Once you are done with the derivation, implement the function <strong><code>innerproduct</code></strong>.</p>

In [52]:
def innerproduct(X,Z=None):
    '''
    function innerproduct(X,Z)
    
    Computes the inner-product matrix.
    Syntax:
    D=innerproduct(X,Z)
    Input:
    X: nxd data matrix with n vectors (rows) of dimensionality d
    Z: mxd data matrix with m vectors (rows) of dimensionality d
    
    Output:
    Matrix G of size nxm
    G[i,j] is the inner-product between vectors X[i,:] and Z[j,:]
    
    call with only one input:
    innerproduct(X)=innerproduct(X,X)
    '''
    if Z is None: # case when there is only one input (X)
        G = innerproduct(X,X)
    else:         # case when there are two inputs (X, Z)
        G = np.dot(X, Z.T)
    
    return(G)

In [53]:
X = np.random.rand(700,10) # define 700 random inputs X
Y = np.random.rand(700,10)
print(X)
X.shape

[[0.2266005  0.64503851 0.34419041 ... 0.77985665 0.97200112 0.07405422]
 [0.29737105 0.04280894 0.51617282 ... 0.78874387 0.30436831 0.70066862]
 [0.21439061 0.6196507  0.55541762 ... 0.9251959  0.40736178 0.79497431]
 ...
 [0.15699322 0.96469839 0.19380465 ... 0.53262119 0.41247507 0.4619085 ]
 [0.53787312 0.02274852 0.98962141 ... 0.63653659 0.36208614 0.14139822]
 [0.42146945 0.03905815 0.24205221 ... 0.65384305 0.05148846 0.10860071]]


(700, 10)

In [61]:
G0 = innerproduct(X,X)
print(G0)


[[3.08806444 1.80933409 2.5174319  ... 2.30833534 1.80175988 1.29546632]
 [1.80933409 2.72439771 2.97049809 ... 1.53798708 1.86751107 2.02905653]
 [2.5174319  2.97049809 3.64456334 ... 2.37669288 2.16044757 2.31188458]
 ...
 [2.30833534 1.53798708 2.37669288 ... 2.27014444 1.33006164 1.19824402]
 [1.80175988 1.86751107 2.16044757 ... 1.33006164 2.51866499 2.09226866]
 [1.29546632 2.02905653 2.31188458 ... 1.19824402 2.09226866 3.06650091]]


In [55]:
G1 = np.dot(X,Y.T)
print(G1)

[[2.09680431 2.57625408 2.06929874 ... 2.24593771 1.79742965 3.14168209]
 [1.76692527 2.30453149 1.91326388 ... 2.15607613 2.016434   3.28923405]
 [2.34904182 2.99390929 2.36522229 ... 2.57138049 2.37962578 3.98337824]
 ...
 [1.8568973  2.21929748 1.83496948 ... 1.82735185 1.75984026 2.73877433]
 [1.86171393 1.71961973 2.25733952 ... 2.53677057 1.55474132 3.10506247]
 [1.8680296  2.05114386 1.65647691 ... 2.11672774 1.60743648 3.43007207]]


In [32]:
#Run this self-test cell to check your code

def innerprod_0():
    # test the output dimensions of innerproduct with one input matrix
    X = np.random.rand(700,10) # define 700 random inputs X
    test = (innerproduct(X).shape==700,700)    # check if inner-product matrix has dimension 700x700
    return test

def innerprod_1():
    # test the output dimensions of innerproduct with two matrices
    X = np.random.rand(700,10) # define 700 random inputs X
    Z = np.random.rand(200,10) # define 200 random inputs Z 
    test=(innerproduct(X,Z).shape ==(700,200)) # check if inner-product matrix has dimensions 700x200
    return test

def innerprod_2():
    X = np.random.rand(700,100) # define 700 random inputs X
    IP1 = innerproduct(X) # compute inner-product matrix with YOUR code
    IP2 = innerproduct_grader(X) # compute inner-product matrix with OUR code
    test = np.linalg.norm(IP1 - IP2) # compute the norm of the difference
    return test<1e-5 # this norm should be essentially 0

def innerprod_3():
    X = np.random.rand(700,100) # define 700 random inputs X
    Z = np.random.rand(300,100) # define 300 random inputs X
    IP1 = innerproduct(X,Z) # compute inner-product matrix with YOUR code
    IP2 = innerproduct_grader(X,Z) # compute inner-product matrix with OUR code
    test = np.linalg.norm(IP1 - IP2) # compute the norm of the difference
    return test<1e-5 # this norm should be essentially 0


runtest(innerprod_0,'innerprod_0 Dimensions with 1 Matrix')
runtest(innerprod_1,'innerprod_1 Dimensions with 2 Matrices')
runtest(innerprod_2,'innerprod_2 Correctness with 1 Matrix')
runtest(innerprod_3,'innerprod_3 Correctness with 2 Matrices')

Running Test: innerprod_0 Dimensions with 1 Matrix ... ✔ Passed!
Running Test: innerprod_1 Dimensions with 2 Matrices ... ✔ Passed!
Running Test: innerprod_2 Correctness with 1 Matrix ... ✔ Passed!
Running Test: innerprod_3 Correctness with 2 Matrices ... ✔ Passed!


In [ ]:
# Autograder test cell - worth 1 point
# runs innerprod_1

In [ ]:
# Autograder test cell - worth 1 Point
# runs innerprod_2

### Exercise 2: Implement `calculate_S` and `calculate_R`

Let us define two new matrices, $S, R \in \mathcal{R}^{n \times m}$
$$
S_ij = \mathbf{x}_i \mathbf{x}_i^\top, \ \ R_{ij} = \mathbf{z}_j \mathbf{z}_j^\top
$$

In the previous activity, we have shown that the _squared_-euclidean matrix $D^2 \in \mathcal{R}^{n \times m}$, defined as
$$
D_{ij}^2 = (\mathbf{x}_i - \mathbf{z}_j)(\mathbf{x}_i - \mathbf{z}_j)^\top,
$$
can be expressed as
$$
D^2 = S + R - 2G.
$$

Later in this exercise, you will implement `l2distance` to calculate $D$. But you will need $S$ and $R$, which you will implement now in **`calculate_S`** and **`calculate_R`** respectively. Ensure that your functions return $S$ and $R$ of size $n \times m$, as they will be added to $-2G$ to get $D^2$.

Think about what the $S$ and $R$ matrices look like. You will find that the values in each row of $S$ and the values in each column of $R$ do not change! This is also apparent when considering that $S_{ij} = \mathbf{x}_i \mathbf{x}_i^\top$ _for all $j$_; similar argument for $R_{ij} = \mathbf{z}_j \mathbf{z}_j^\top$ _for all $i$_.
$$
S = \begin{bmatrix}
\mathbf{x}_1 \mathbf{x}_1^\top & \mathbf{x}_1 \mathbf{x}_1^\top & \cdots & \mathbf{x}_1 \mathbf{x}_1^\top\\
\mathbf{x}_2 \mathbf{x}_2^\top & \mathbf{x}_2 \mathbf{x}_2^\top & \cdots & \mathbf{x}_2 \mathbf{x}_2^\top\\
\vdots & \vdots & \ddots & \vdots\\
\mathbf{x}_n \mathbf{x}_n^\top & \mathbf{x}_n \mathbf{x}_n^\top & \cdots & \mathbf{x}_n \mathbf{x}_n^\top\\
\end{bmatrix}, \ 
R = \begin{bmatrix}
\mathbf{z}_1 \mathbf{z}_1^\top & \mathbf{z}_2 \mathbf{z}_2^\top & \cdots & \mathbf{z}_m \mathbf{z}_m^\top\\
\mathbf{z}_1 \mathbf{z}_1^\top & \mathbf{z}_2 \mathbf{z}_2^\top & \cdots & \mathbf{z}_m \mathbf{z}_m^\top\\
\vdots & \vdots & \ddots & \vdots\\
\mathbf{z}_1 \mathbf{z}_1^\top & \mathbf{z}_2 \mathbf{z}_2^\top & \cdots & \mathbf{z}_m \mathbf{z}_m^\top\\
\end{bmatrix}.
$$

Now you just need to figure out how to calculate $\mathbf{x}_i \mathbf{x}_i^\top$ and $\mathbf{z}_j \mathbf{z}_j^\top$ without loops. You might find the fact $\mathbf{a} \mathbf{a}^\top = \sum_{k=1}^d a_k^2$ and repeat function [`np.repeat`](https://numpy.org/doc/stable/reference/generated/numpy.repeat.html) (and its `axis` parameter) useful.

In [128]:
def calculate_S(X, n, m):
    '''
    function calculate_S(X)
    
    Computes the S matrix.
    Syntax:
    S=calculate_S(X)
    Input:
    X: nxd data matrix with n vectors (rows) of dimensionality d
    n: number of rows in X
    m: output number of columns in S
    
    Output:
    Matrix S of size nxm
    S[i,j] is the inner-product between vectors X[i,:] and X[i,:]
    '''
    assert n == X.shape[0]
    
    S = np.einsum('ij,ij->i', X, X)[:, np.newaxis]
    
    print(S)

In [113]:
X = np.random.rand(700,100) # define random inputs
Z = np.random.rand(800,100) # define random inputs

In [114]:
n,d1=X.shape
m,d2=Z.shape

In [119]:
S = np.dot(X, X.T)
S.shape

(700, 700)

In [121]:
n = S.shape[0]
m = S.shape[1]

In [127]:
S = np.einsum('ij,ij->i', X, X)[:, np.newaxis]
S.shape

(700, 1)

In [126]:
R = np.einsum('ij,ij->i', Z, Z)[:, np.newaxis]
R.shape

(800, 1)

In [132]:
def calculate_R(Z, n, m):
    '''
    function calculate_R(Z)
    
    Computes the R matrix.
    Syntax:
    R=calculate_R(Z)
    Input:
    Z: mxd data matrix with m vectors (rows) of dimensionality d
    n: output number of rows in Z
    m: number of rows in Z
    
    Output:
    Matrix R of size nxm
    R[i,j] is the inner-product between vectors Z[j,:] and Z[j,:]
    '''
    assert m == Z.shape[0]
    
    R = np.einsum('ij,ij->i', Z, Z)[:, np.newaxis]
    
    print(R)

In [133]:
#Run this self-test cell to check your code

def calculate_S_dimensions():
    X = np.random.rand(700,100) # define random inputs
    Z = np.random.rand(800,100) # define random inputs
    n,d1=X.shape
    m,d2=Z.shape    
    S1 = calculate_S(X, n, m) # compute distances from your solutions
    o1,o2=S1.shape
    return (o1==n) and (o2==m)

def calculate_S_accuracy():
    X = np.random.rand(700,100) # define random inputs
    S1 = calculate_S(X, X.shape[0], 800) # compute distances from your solutions
    S2 = calculate_S_grader(X, X.shape[0], 800) #compute distance from ground truth
    test = np.linalg.norm(S1 - S2) # compare the two
    return test<1e-5 # difference should be small

def calculate_R_dimensions():
    X = np.random.rand(700,100) # define random inputs
    Z = np.random.rand(800,100) # define random inputs
    n,d1=X.shape
    m,d2=Z.shape    
    R1 = calculate_R(Z, n, m) # compute distances from your solutions
    o1,o2=R1.shape
    return (o1==n) and (o2==m)

def calculate_R_accuracy():
    Z = np.random.rand(800,100) # define random inputs
    R1 = calculate_R(Z, 700, Z.shape[0]) # compute distances from your solutions
    R2 = calculate_R_grader(Z, 700, Z.shape[0]) #compute distance from ground truth
    test = np.linalg.norm(R1 - R2) # compare the two
    return test<1e-5 # difference should be small

runtest(calculate_S_dimensions,'calculate_S_dimensions')
runtest(calculate_S_accuracy,'calculate_S_accuracy')
runtest(calculate_R_dimensions,'calculate_R_dimensions')
runtest(calculate_R_accuracy,'calculate_R_accuracy')

Running Test: calculate_S_dimensions ... [[29.30138007]
 [31.30407978]
 [33.34827621]
 [29.71983461]
 [31.44109399]
 [36.42644059]
 [30.80930544]
 [31.47845259]
 [31.83518347]
 [26.07829275]
 [35.17857071]
 [38.93978821]
 [25.36960814]
 [34.14404902]
 [30.84170886]
 [32.92135714]
 [38.712993  ]
 [33.06241197]
 [34.69212072]
 [32.19487265]
 [32.33712663]
 [32.56019739]
 [31.57792013]
 [30.11816083]
 [29.50562912]
 [35.65326486]
 [30.38149864]
 [31.91669548]
 [29.39117132]
 [34.70853824]
 [31.50839693]
 [31.78941064]
 [33.68372514]
 [36.42035275]
 [37.85505092]
 [34.01246063]
 [30.78735609]
 [34.75067656]
 [36.31875886]
 [35.7335541 ]
 [40.32161964]
 [34.21633885]
 [36.06316685]
 [34.44700143]
 [35.2389606 ]
 [30.18894746]
 [33.82250844]
 [29.47047465]
 [29.63243245]
 [27.27254517]
 [35.13998977]
 [32.14233233]
 [38.53622653]
 [34.27839095]
 [36.99735304]
 [31.75768851]
 [30.03979903]
 [32.76077253]
 [30.30206439]
 [30.92219234]
 [34.08576788]
 [38.42426798]
 [37.41614807]
 [37.49032311]

<h3>Exercise 3: Implement <code>l2distance</code></h3>

$$
D^2_{ij}=(\mathbf{x}_i-\mathbf{z}_j)(\mathbf{x}_i-\mathbf{z}_j)^\top, \ D^2 = S + R - 2G
$$
    
<p> In this exercise, you will use the above formula to implement the function <strong><code>l2distance</code></strong>, which computes the Euclidean distance matrix $D$ without a single loop. </p>

<p><strong>Hint</strong>: Make sure that when you take the square root of the squared distance matrix, ensure that all entries are non-negative. Sometimes very small positive numbers can become negative due to numerical imprecision. Knowing that all distances must always be non-negative, you can simply overwrite all negative values as 0.0 to avoid unintended consequences </p>

In [ ]:
def l2distance(X,Z=None):
    '''
    function D=l2distance(X,Z)
    
    Computes the Euclidean distance matrix.
    Syntax:
    D=l2distance(X,Z)
    Input:
    X: nxd data matrix with n vectors (rows) of dimensionality d
    Z: mxd data matrix with m vectors (rows) of dimensionality d
    
    Output:
    Matrix D of size nxm
    D(i,j) is the Euclidean distance of X(i,:) and Z(j,:)
    
    call with only one input:
    l2distance(X)=l2distance(X,X)
    '''
    if Z is None:
        Z=X;

    n,d1=X.shape
    m,d2=Z.shape
    assert (d1==d2), "Dimensions of input vectors must match!"

    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
#Run this self-test cell to check your code

def distance_accuracy(): 
    X = np.random.rand(700,100) # define random inputs
    D1 = l2distance(X) # compute distances from your solutions
    D2 = l2distance_grader(X) #compute distance from ground truth
    test = np.linalg.norm(D1 - D2) # compare the two
    return test<1e-5 # difference should be small

def distance_squareroot():  
    X = np.random.rand(700,100) # define random inputs
    D1 = l2distance(X) # compute distances from your solutions
    D2sq = l2distance_grader(X)**2 #compute distance from ground truth *but square them*
    test = np.linalg.norm(D1 - D2sq) # compare the two
    return test>1e-5 # difference should be big

def dimensions():
    X = np.random.rand(700,100) # define random inputs
    Z = np.random.rand(800,100) # define random inputs
    n,d1=X.shape
    m,d2=Z.shape    
    D1 = l2distance(X,Z) # compute distances from your solutions
    o1,o2=D1.shape
    return (o1==n) and (o2==m)

def matrix_dist_accuracy():
    X = np.random.rand(700,100)
    Z = np.random.rand(300,100)
    D1Z = l2distance(X,Z)
    D2Z = l2distance_grader(X,Z)
    test = np.linalg.norm(D1Z - D2Z)
    return test<1e-5

runtest(distance_accuracy,'distance_accuracy')
runtest(distance_squareroot,'distance_squareroot')
runtest(dimensions,'dimensions')
runtest(matrix_dist_accuracy,'matrix_dist_accuracy')

In [ ]:
# Autograder test cell - worth 1 Point
# runs distance_accuracy

In [ ]:
# Autograder test cell - worth 1 Point
# runs distance_squareroot

In [ ]:
# Autograder test cell - worth 1 Point
# runs dimensions

In [ ]:
# Autograder test cell - worth 1 Point
# runs matrix_dist_accuracy

Let's now compare the speed of your l2-distance function against the previous naïve implementation:

In [ ]:
import time
current_time = lambda: int(round(time.time() * 1000))

X=np.random.rand(700,100)
Z=np.random.rand(300,100)

print("Running the naïve version...")
before = current_time()
Dslow=l2distanceSlow(X)
after = current_time()
t_slow = after - before
print("{:2.0f} ms".format(t_slow))

print("Running the vectorized version...")
before = current_time()
Dfast=l2distance(X)
after = current_time()
t_fast = after - before
print("{:2.0f} ms".format(t_fast))


speedup = t_slow / t_fast
print("The two methods should deviate by very little: {:05.6f}".format(norm(Dfast-Dslow)))
print("but your NumPy code was {:05.2f} times faster!".format(speedup))

How much faster is your code now? With this implementation, you should easily be able to compute the distances between <strong>many more</strong> vectors. It should be clear now, even for small datasets, that the for-loop based implementation could take several days or even weeks to perform basic operations that take seconds or minutes with well-written NumPy code.